# Lecture 9 - Policy Gradient II

provided by [Stanford CS234](https://www.youtube.com/watch?v=FgzM3zpZ55o)

---

<div class="alert alert-block alert-info">
Table of Contents: <br>
    
<ul>
    <li>1. <a href="#1.-Introduction">Introduction</a>
    <li>2. <a href="#2.-%22Vanilla%22-Policy-Gradient-Algorithm">"Vanilla" Policy Gradient Algorithm</a></li>
    <li>3. <a href="#3.-Need-for-Automatic-Step-Size-Tuning">Need for Automatic Step Size Tuning</a>
    <ul>
        <li>3.1. <a href="#3.1.-Local-Approximation">Local Approximation</a></li>
    </ul>
    </li>
    <li>4. <a href="#4.-Resource">Resource</a></li>
</ul>
</div>

# 1. Introduction

For Policy Gradient algorithms, we want to converge as fast as possible to the local optima as well as have monotonic improvement.

Last lecture, we focused on policy-based methods. This lecture we focus on policy and value-based methods which are commonly referred to as __actor-critic__ methods.

# 2. "Vanilla" Policy Gradient Algorithm

Initialize policy parameter $\theta$, baseline $b$ <br>
for iteration=1, 2, ..., do <br>
$\quad$ Collect a set of trajectories by executing the current policy <br>
$\quad$ At each timestep $t$ in each trajectory $\tau^{i}$ <br>
$\quad\quad$ Compute Return $G_{t}^{i} = \sum_{t' = t}^{T - 1}r_{t}^{i}$, and <br>
$\quad\quad$ Advantage estimate $\hat{A}_{t}^{i} = G_{t}^{i} - b(s_{t})$. <br>
$\quad$ Re-fit the baseline, by minimizing $\sum_{i}\sum_{t}||b(s_{t}) - G_{t}^{i}||^{2}$, <br>
$\quad$ Update the policy, using a policy gradient estimate $\hat{g}$, <br>
$\quad\quad$ Which is a sum of terms $\nabla_{\theta} log \pi(a_{t}~|~s_{t}, \theta) \hat{A}_{t}$. <br>
$\quad\quad$ Plug $\hat{g}$ into SGD or ADAM <br>
<br>

_Algorithm 1. "Vanilla" Policy Gradient Algorithm._

The __"Vanilla" Policy Gradient__ algorithm is a general skeleton or framework for many different PG methods. REINFORCE is a prime example of this template. Notice that this algorithm uses the equations we defined in understanding likelihood policies. The only new idea introduced here is the baseline.

$b(s_{t})$ is simply a function (e.g. deep/shallow neural network, etc) that takes in a state and outputs an expected return. As this "vanilla" PG algorithm iterates, the baselineshould be continuously re-fit to perfectly match the expected return (undiscounted).

We introduce a baseline into our standard PG algorithm template because it reduces variance.

$$
\begin{equation}
    \begin{split}
\nabla_{\theta} V(\theta) & = \frac{1}{m} \sum_{i = 1}^{m} R(\tau^{(i)}) \sum_{t = 0}^{T_{i}} \nabla_{\theta} log \pi_{\theta}(a_{t}~|~s_{t}) \hspace{1em} (Eq.~1)\\
    & = \mathbb{E}_{\tau}[\sum_{t = 0}^{T - 1} \nabla_{\theta} log \pi_{\theta}(a_{t}~|~s_{t}) G_{t}^{(i)}]\\
    \end{split}
\end{equation}
$$

Eq. 1 are our standard gradient formulas from the previous lecture. 

$$
\hat{A}_{t}^{i} = G_{t}^{i} - b(s_{t})\\
\nabla_{\theta} V(\theta) = \mathbb{E}_{\tau}[\sum_{t = 0}^{T - 1} \nabla_{\theta} log \pi_{\theta}(a_{t}~|~s_{t}) \hat{A}_{t}^{i}] \hspace{1em} (Eq.~2)\\
$$

Eq. 2 is the same equation written with the baseline included.

Now notice we can substitute $V$ or $Q$ into the advantage function. Additionally, we can have a method that learns this value function. We call this a __critic__. Instead of just the sum of future rewards $G_{t}^{i}$, we can use the $Q$ function. We can use TD or MC methods to compute that reward.

$$
\hat{A}_{t}^{i} = Q(s_{t}, w) - b(s_{t})\\
$$

But wait! Keep in mind this algorithm so far is policy-based. To make it both policy and value-based, we can parameterize $R(\tau^{(i)})$.

# 3. Need for Automatic Step Size Tuning

At each iteration of our "vanilla" PG algorithm, we want the value function for the new updated policy to be better than the previous iteration's policy: $V^{\pi'} \ge V^{pi}$.

Why is the step size important in this scenario? Well, the step size affects how we converge and how fast we do it. If we have a bad step size, our policy will be updated a certain way, and consequently, it will collect data in a biased way.

So what are some ways to account for this issue?
* simple step size with line search
    * simple but expensive 
    * naive
* auto-step-size selection
    * can we ensure the current policy's value function is greater than or equal to the previous iteration's policy's value function?

$$
V(\theta) = \mathbb{E}_{\pi_{\theta}}[\sum_{t = 0}^{\infty}\gamma^{t} R(s_{t}, a_{t}); \pi_{\theta}] \hspace{1em} (Eq.~3)\\
$$

Eq. 3 says we want to maximize the value function for a given policy in the infinite horizon setting.

We can decompose this function into parts: 

$$
\begin{equation}
    \begin{split}
    L_{\pi}(\tilde{\pi}) = V(\tilde{\theta}) & = V(\theta) + \mathbb{E}_{\pi_{\tilde{\theta}}}[\sum_{t = 0}^{\infty} \gamma^{t} A_{\pi}(s_{t}, a_{t})]\\
    & = V(\theta) + \sum_{s} \mu_{\tilde{\pi}}(s) \sum_{a} \tilde{\pi}(a~|~s) A_{\pi}(s, a)\\
    \mu_{\tilde{\pi}}(s) & = \mathbb{E}_{\tilde{\pi}}[\sum_{t = 0}^{\infty} \gamma^{t} I(s_{t} = s)]
    \end{split}
\end{equation} \hspace{1em} (Eq.~4)\\
$$

Notice how the first and second equations of Eq. 4 are the exact same just written in different ways. 

The only new idea in these equations is the tilde (~). $\tilde{\pi}$ is the new policy (at iteration $i + 1$) and the same goes for $\tilde{\theta}$. 

So we understand this is for auto step size tuning, but we don't know what $\mu_{\tilde{\pi}}$. Well to be more specific, we can't calculate it just yet (it requires the new policy at iteration $i + 1$). How do we fix this?

There are a few approaches to fixing this issue:
* __local approximation__
* __trust regions__
* __TRPO algorithm__

## 3.1. Local Approximation

We can slightly rewrite Eq. 4 so that we have a substitute for $\mu_{\tilde{\pi}}$:

$$
L_{\pi}(\tilde{\pi}) = V(\theta) + \sum_{s} \mu_{\pi}(s) \sum_{a} \tilde{\pi}(a~|~s) A_{\pi}(s, a) \hspace{1em} (Eq.~5)\\
$$

Eq. 5, instead  of using the discounted weighted frequency of state $s$ under policy $\mu_{\tilde{\pi}}$, uses $\mu_{\pi}$, the current policy's discounted weighted frequency of state $s$

This begs the question: how do Eq. 3 and Eq. 4 fit into our current understanding of policy gradients? Over Lecture 8 and Lecture 9, we have seen a lot of formulas involving value functions.

For now, I'm still not too sure. Let's give it some time.

My conclusion is that we formulate our objective function like this (there are many other ways to do it) because we want to find a sure-fire way to have monotonic improvement in gradient-based policy search.

# 4. Resource

If you missed the link right below the title, I'm providing the resource here again along with the course website.

- [Stanford CS234](https://www.youtube.com/watch?v=FgzM3zpZ55o)
- [Course Website](http://web.stanford.edu/class/cs234/index.html)

This is a series of 15 lectures provided by Stanford.
